In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import glob
import seaborn as sns

## Read from `~/metrics/epoch_metrics/` directory

In [48]:
df = pd.read_parquet("../../metrics/epoch_metrics/")

# Hyper-Parameters

## All tests summary

Note that this is a summary. 

Notes:
1. `synchronous` method implies `theta` equal to `0.0` and the reverse.
2. The stopping criterion (epochs at the moment) is not the same for all combination, in fact, it changes. For `LeNet-5` currently all tests end at 50 epochs. For the `AdvancedCNN` we go up to 350 epochs in some tests and some others up until 250.
3. `Theta` greater than 2 are exploratory tests and will not be considered at all later on.

In [49]:
df_lenet = df[df['nn_name'] == 'LeNet-5']

In [50]:
for col in ['dataset_name', 'fda_name', 'nn_name', 'num_clients', 'batch_size', 'num_steps_until_rtc_check', 'theta', 'bias']:
    print(f"{col}: {sorted(list(df_lenet[col].unique()))}")

dataset_name: ['MNIST']
fda_name: ['linear', 'naive', 'sketch', 'synchronous']
nn_name: ['LeNet-5']
num_clients: [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
batch_size: [32, 64, 128, 256]
num_steps_until_rtc_check: [1]
theta: [0.0, 0.5, 1.0, 1.5, 2.0, 3.0, 5.0, 7.0, 10.0, 12.0, 15.0]
bias: [nan, 0.3, 0.6, 0.9]


## Query All tests

In [51]:
test_combinations = df.groupby(['dataset_name', 'nn_name', 'fda_name', 'num_steps_until_rtc_check', 'batch_size', 'theta', 'num_clients'])['epoch'].max().reset_index()

In [52]:
test_combinations[
    (test_combinations['nn_name'] == 'AdvancedCNN') &
    (test_combinations['fda_name'] == 'naive') &
    (test_combinations['theta'] == 20.0) &
    (test_combinations['num_clients'] == 5)
]

,dataset_name,nn_name,fda_name,num_steps_until_rtc_check,batch_size,theta,num_clients,epoch
278,MNIST,AdvancedCNN,naive,1,32,20.0,5,100


# Helpful new Dataframe metrics

### Add Helpful Dataset Metrics 

In [53]:
def dataset_n_train(row):
    if row['dataset_name'] == "MNIST":
        return 60_000
    else:
        return -1


def dataset_one_sample_bytes(row):
    if row['dataset_name'] == "MNIST":
        # input image 784 tf.float32 pixels and a tf.int32 label
        return 4 * (784 + 1)
    else:
        return -1

In [54]:
df['n_train'] = df.apply(dataset_n_train, axis=1)
df['one_sample_bytes'] = df.apply(dataset_one_sample_bytes, axis=1)

### Add Helpful model metrics

In [55]:
df['model_bytes'] = df['nn_num_weights'] * 4

### Add Helpful FDA method metrics

In [56]:
def fda_local_state_bytes(row):
    if row['fda_name'] == "naive":
        return 4
    if row['fda_name'] == "linear":
        return 8
    if row['fda_name'] == "sketch":
        return row['sketch_width'] * row['sketch_depth'] * 4 + 4
    if row['fda_name'] == "synchronous":
        return 0
    if row['fda_name'] == 'gm':
        return 0.125  # one bit

In [57]:
df['local_state_bytes'] = df.apply(fda_local_state_bytes, axis=1)

### Add Total Steps

total steps (a single fda step might have many normal SGD steps, batch steps)

In [58]:
df['total_steps'] = df['total_fda_steps'] * df['num_steps_until_rtc_check']

### Add communication metrics

The communication bytes exchanged for model synchronization. Remember that the Clients send their models to the Server and the Server sends the global model back. This happens at the end of every round.

In [59]:
df['model_bytes_exchanged'] = df['total_rounds'] * df['model_bytes'] * df['num_clients'] * 2

The communication bytes exchanged for monitoring the variance. This happens at the end of every FDA step which consists of `num_steps_until_rtc_check` number of steps. 

In [60]:
df['monitoring_bytes_exchanged'] = df['local_state_bytes'] * df['total_fda_steps'] * df['num_clients']

The total communication bytes exchanged in the whole Federated Learning lifecycle.

In [61]:
df['total_communication_bytes'] = df['model_bytes_exchanged'] + df['monitoring_bytes_exchanged']

In [62]:
df['total_communication_gb'] = df['total_communication_bytes'] / 10**9

Add rounds in one epoch.

In [63]:
df = df.sort_values(by=['dataset_name', 'fda_name', 'nn_num_weights', 'num_clients', 'batch_size', 'num_steps_until_rtc_check', 'theta', 'epoch'])

df['epoch_rounds'] = df.groupby(['dataset_name', 'fda_name', 'nn_num_weights', 'num_clients', 'batch_size', 'num_steps_until_rtc_check', 'theta'])['total_rounds'].diff()

# NaN first epoch
df['epoch_rounds'] = df['epoch_rounds'].fillna(df['total_rounds'])

df['epoch_rounds'] = df['epoch_rounds'].astype(int)

# HyperParameter ranking

### LeNet-5
On 2 `Nvidia A10`:
1. Batch Size = 32 -> 6.613 ms ± 0.128 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 7.509 ms ± 0.065 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 8.02 ms ± 0.099 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 9.258 ms ± 0.336 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## AdvancedCNN

On 2 `Nvidia A10`:
1. *Batch Size* = 32 -> 8.853 ms ± 0.0917 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 10.325 ms ± 0.215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 11.989 ms ± 0.134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 16.47 ms ± 0.294 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [64]:
step_ms = {
    ("AdvancedCNN", 32): 8.853,
    ("AdvancedCNN", 64): 10.325,
    ("AdvancedCNN", 128): 11.989,
    ("AdvancedCNN", 256): 16.47,
    ("LeNet-5", 32): 6.613,
    ("LeNet-5", 64): 7.509,
    ("LeNet-5", 128): 8.02,
    ("LeNet-5", 256): 9.258
}

In [65]:
import numpy as np

def cpu_time_cost(row):
    """ Total cpu time cost in (sec).
    A single `step` means each client performed a single `step` 
    """
    return row['total_steps'] * step_ms[(row['nn_name'], row['batch_size'])] / 1000

def communication_time_cost(num_clients, total_communication_bytes, comm_model):
    """ Assuming channel is 1Gbps """

    total_communication_gbit = total_communication_bytes * 8e-9

    if comm_model == 'common_channel':
        
        return ((num_clients - 1) / num_clients) * total_communication_gbit    # sec

    if comm_model == 'hypercube':

        return (np.ceil(np.log(num_clients)) / num_clients) * total_communication_gbit   # sec

In [66]:
df['cpu_time_cost'] = df.apply(cpu_time_cost, axis=1)

In [67]:
df['hypercube_communication_time_cost'] = communication_time_cost(df['num_clients'], df['total_communication_bytes'], 'hypercube')

In [68]:
df['common_channel_communication_time_cost'] = communication_time_cost(df['num_clients'], df['total_communication_bytes'], 'common_channel')

In [69]:
df['hypercube_time_cost'] = df['cpu_time_cost'] + df['hypercube_communication_time_cost']

In [70]:
df['common_channel_time_cost'] = df['cpu_time_cost'] + df['common_channel_communication_time_cost']

In [71]:
df['hypercube_comm_cpu_time_ratio'] = df['hypercube_communication_time_cost'] / df['cpu_time_cost']

In [72]:
df['common_channel_comm_cpu_time_ratio'] = df['common_channel_communication_time_cost'] / df['cpu_time_cost']

# Plots about cost

In [73]:
# Define styles for each fda_name
fda_styles = {
    'naive': 'o-r',
    'linear': 's-g',
    'sketch': '^-b',
    'synchronous': 'x-c'
}
fda_names = sorted(df['fda_name'].unique())

In [74]:
import matplotlib

num_clients_values = sorted(df['num_clients'].unique())
cmap = matplotlib.colormaps['tab20b']
colors_dict = {
    num_clients: color 
    for num_clients, color in zip(num_clients_values, cmap(np.linspace(0, 1, len(num_clients_values))))
}

## KDE Helper

In [75]:
sns_params = {
    'bw_method': 'scott',
    'bw_adjust': 0.7,
    'fill': True,
    'alpha': 0.375
}

sns_params_biases = {
    'bw_method': 'scott',
    'bw_adjust': 0.7,
    'fill': False,
    'alpha': 0.8
}

base_colors = {
    'linear': 'blue',
    'naive': 'orange',
    'sketch': 'green',
    'synchronous': 'red'
}

In [76]:
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 14

## Total time cost with accuracy

### KDE

In [77]:
def kde_time_cost(df, filename, x_log=True):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    plt.rcParams['font.size'] = 20
    plt.rcParams['legend.fontsize'] = 12
    
    pdf = PdfPages(filename)
    
    fig, axs = plt.subplots(1, 2, figsize=(20, 8))

    # Prepare a list to store the average communication for each method
    avg_time_cost_common_channel_dict = {}
    avg_time_cost_hypercube_dict = {}
    
    # Prepare lists to store the average information (string) for each subplot
    avg_info_common_channel = []
    avg_info_hypercube = []

    for fda_name in fda_names:
        common_channel_df = df[df['fda_name'] == fda_name]['common_channel_time_cost']
        avg_time_cost_common_channel_dict[fda_name] = common_channel_df.mean()
        avg_info_common_channel.append(f'{fda_name}: {avg_time_cost_common_channel_dict[fda_name]:.2f} sec')
        sns.kdeplot(common_channel_df, label=fda_name, ax=axs[0], log_scale=log_scale, **sns_params)
        
        hypercube_df = df[df['fda_name'] == fda_name]['hypercube_time_cost']
        avg_time_cost_hypercube_dict[fda_name] = hypercube_df.mean()
        avg_info_hypercube.append(f'{fda_name}: {avg_time_cost_hypercube_dict[fda_name]:.2f} sec')
        sns.kdeplot(hypercube_df, label=fda_name, ax=axs[1], log_scale=log_scale, **sns_params)
        
    
    """
    text = "Average Time Cost:\n" + '\n'.join(avg_info_common_channel)
    # Add the text annotation inside the plot
    axs[0].text(0.78, 0.68, text, transform=axs[0].transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
    """
    
    if not x_log:
        axs[0].set_xlim(left=0)
    axs[0].set_xlabel('Time Cost (sec)')
    axs[0].set_ylabel('Density')
    axs[0].legend()
    axs[0].set_title("Common Channel Communication Model")
    axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[0].legend()

    """
    text = "Average Time Cost:\n" + '\n'.join(avg_info_hypercube)
    # Add the text annotation inside the plot
    axs[1].text(0.79, 0.68, text, transform=axs[1].transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
    """
    
    if not x_log:
        axs[1].set_xlim(left=0)
        
    axs[1].set_xlabel('Time Cost (sec)')
    axs[1].set_ylabel('Density')
    axs[1].legend()
    axs[1].set_title("Hypercube Communication Model")
    axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[1].legend()
    
    plt.tight_layout()
    
    pdf.savefig(fig)
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close(fig)
    pdf.close()
    
    plt.rcParams['font.size'] = 14
    plt.rcParams['legend.fontsize'] = 12

In [78]:
def kde_time_cost_biases(df, filename, x_log=False):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    plt.rcParams['font.size'] = 20
    plt.rcParams['legend.fontsize'] = 12
    
    pdf = PdfPages(filename)
    
    fig, axs = plt.subplots(1, 2, figsize=(20, 8))

    for fda_name in fda_names:
        
        df_fda_method = df[df['fda_name'] == fda_name]
        
        biases = sorted(df_fda_method['bias'].unique(), reverse=True)[::-1]
        
        color_palette = sns.light_palette(base_colors[fda_name], n_colors=len(biases)+1)[::-1]
        
        for i, bias in enumerate(biases): #HERE
            if pd.isna(bias):
                mask = df_fda_method['bias'].isna()
            else:
                mask = df_fda_method['bias'] == bias
                
            df_bias = df_fda_method[mask]
            
            if df_bias.empty:
                continue

            common_channel_df = df_bias['common_channel_time_cost']
            sns.kdeplot(common_channel_df, label=f'{fda_name} w/ bias {bias}', ax=axs[0], color=color_palette[i], log_scale=log_scale, **sns_params_biases)

            hypercube_df = df_bias['hypercube_time_cost']
            sns.kdeplot(hypercube_df, label=f'{fda_name} w/ bias {bias}', ax=axs[1], color=color_palette[i], log_scale=log_scale, **sns_params_biases)

    
    if not x_log:
        axs[0].set_xlim(left=0)
    axs[0].set_xlabel('Time Cost (sec)')
    axs[0].set_ylabel('Density')
    axs[0].legend()
    axs[0].set_title("Common Channel Communication Model")
    axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[0].legend()
    
    if not x_log:
        axs[1].set_xlim(left=0)
        
    axs[1].set_xlabel('Time Cost (sec)')
    axs[1].set_ylabel('Density')
    axs[1].legend()
    axs[1].set_title("Hypercube Communication Model")
    axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[1].legend()
    
    plt.tight_layout()
    
    pdf.savefig(fig)
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close(fig)
    pdf.close()
    
    plt.rcParams['font.size'] = 14
    plt.rcParams['legend.fontsize'] = 12

## Total Communication cost (in gb) with accuracy (scatter)

### KDE

In [79]:
def kde_communication_cost(df, filename, x_log=True):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    pdf = PdfPages(filename)
    
    plt.figure(figsize=(10, 6))

    avg_communications_dict = {}
    avg_info = []

    for fda_name in fda_names:
        fda_data_df = df[df['fda_name'] == fda_name]['total_communication_gb']
        avg_communications_dict[fda_name] = fda_data_df.mean()
        avg_info.append(f'{fda_name}: {avg_communications_dict[fda_name]:.2f} GB')
        
        # Plotting only the KDE using kdeplot
        sns.kdeplot(fda_data_df, label=fda_name, log_scale=log_scale, **sns_params)

    
    text = "Average Communication:\n" + '\n'.join(avg_info)
    # Add the text annotation inside the plot
    plt.text(0.02, 0.97, text, transform=plt.gca().transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
    

    if not x_log:
        plt.xlim(left=0)
        
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    plt.xlabel('Communication (GB)')
    plt.ylabel('Density')
    plt.legend()
    plt.tight_layout()
    
    pdf.savefig(plt.gcf()) # Save the current figure
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close()
    pdf.close()

## Total CPU time (in Seconds) with accuracy

### KDE

In [80]:
def kde_cpu_time_cost(df, filename, x_log=False):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    pdf = PdfPages(filename)
    
    plt.figure(figsize=(10, 6))

    avg_cpu_dict = {}
    avg_info = []

    for fda_name in fda_names:
        fda_data_df = df[df['fda_name'] == fda_name]['cpu_time_cost']
        avg_cpu_dict[fda_name] = fda_data_df.mean()
        avg_info.append(f'{fda_name}: {avg_cpu_dict[fda_name]:.2f} sec')
        
        # Plotting only the KDE using kdeplot
        sns.kdeplot(fda_data_df, label=fda_name, log_scale=log_scale, **sns_params)

    """
    text = "Average CPU cost:\n" + '\n'.join(avg_info)
    # Add the text annotation inside the plot
    plt.text(0.81, 0.7, text, transform=plt.gca().transAxes, fontsize=9, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
    """

    if not x_log:
        plt.xlim(left=0)
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    plt.xlabel('CPU time cost (sec)')
    plt.ylabel('Density')
    plt.legend()
    plt.tight_layout()
    
    pdf.savefig(plt.gcf()) # Save the current figure
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close()
    pdf.close()

## Communication/CPU time - Accuracy

### Scatter

In [81]:
def scatter_time_cost_cpu_ratio(df, filename):
    
    pdf = PdfPages(filename)

    fig, axs = plt.subplots(1, 2, figsize=(20, 6))

    # Plot the data points for each method (fda_name) for single-bus model
    for fda_name in fda_names:
        fda_filtered_data = df[(df['fda_name'] == fda_name)] 
        axs[0].scatter(fda_filtered_data['common_channel_comm_cpu_time_ratio'], fda_filtered_data['accuracy'], label=fda_name)

    axs[0].set_xlabel('(Communication time) / (CPU time)')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend()
    axs[0].set_title("Common Channel Communication Model")

    # Plot the data points for each method (fda_name) for multi-bus model
    for fda_name in fda_names:
        fda_filtered_data = df[(df['fda_name'] == fda_name)] 
        axs[1].scatter(fda_filtered_data['hypercube_comm_cpu_time_ratio'], fda_filtered_data['accuracy'], label=fda_name)

    axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)   
        
    axs[1].set_xlabel('(Communication time) / (CPU time)')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()
    axs[1].set_title("Hypercube Communication Model")
    
    axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

    plt.tight_layout()
    pdf.savefig(fig)
        
    # Close the current figure to prevent it from being displayed in the notebook
    plt.close(fig)
    pdf.close()

## Different FDA-method runs visualization with lines (per clients)

In [82]:
import matplotlib
from matplotlib import cm

def fda_method_run_line_plot(df, fda_name, filename, limit_x_axis=False):
    
    hybrid_df = df[df.fda_name == fda_name]
    
    batch_size_values = sorted(hybrid_df['batch_size'].unique())
    theta_values = sorted(hybrid_df['theta'].unique())
    
    pdf = PdfPages(filename)

    for batch_size in batch_size_values:
        for theta in theta_values:
            filtered_data = hybrid_df[(hybrid_df['theta'] == theta) &
                                       (hybrid_df['batch_size'] == batch_size)] 

            if filtered_data.empty:
                continue
            
            num_clients_values = sorted(filtered_data['num_clients'].unique())
            
            #print(f"{batch_size} {theta} {filtered_data[['fda_name','accuracy']]}")
                
            # Because num_clients = 2 reaches very good accuracy very early we need to limit the x-axis so we can 
            # visualize the rest of the data more easily. We get the next max time cost and limit x
            # Get the maximum 'common_channel_time_cost' and 'hypercube_time_cost' when num_clients is not 2
            if limit_x_axis:
                # The num clients = 2 cause problems. We put limit 10 after the next in line
                max_common_channel_time_cost = filtered_data[filtered_data['num_clients'] != 2]['common_channel_time_cost'].max()
                max_hypercube_time_cost = filtered_data[filtered_data['num_clients'] != 2]['hypercube_time_cost'].max()


            fig, axs = plt.subplots(1, 2, figsize=(20, 6))

            # Plot each group with a unique color based on num_clients
            for num_clients in num_clients_values:
                data = filtered_data[filtered_data['num_clients'] == num_clients]

                axs[0].plot(data['common_channel_time_cost'], data['accuracy'], color=colors_dict[num_clients], label=num_clients, marker='o', markersize=3)
                axs[1].plot(data['hypercube_time_cost'], data['accuracy'], color=colors_dict[num_clients], label=num_clients, marker='o', markersize=3)
                
            
            axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            #axs[0].set_ylim(top=1)
            #axs[1].set_ylim(top=1)

            axs[0].set_xlabel('Time Cost')
            axs[0].set_ylabel('Accuracy')
            axs[0].set_title("Common Channel Communication Model")
            axs[0].legend(title='Num Clients')
            if limit_x_axis:
                axs[0].set_xlim(0, max_common_channel_time_cost+700)  # set x-axis limit

            axs[1].set_xlabel('Time Cost')
            axs[1].set_ylabel('Accuracy')
            axs[1].set_title("Hypercube Communication Model")
            axs[1].legend(title='Num Clients')
            if limit_x_axis:
                axs[1].set_xlim(0, max_hypercube_time_cost+500)  # set x-axis limit

            title = f'Batch Size : {batch_size} , $\Theta$ : {theta}'

            fig.suptitle(title)

            plt.tight_layout()

            pdf.savefig(fig)

            plt.close(fig)

    pdf.close()

## Keep Hyper-parameters fixed and plot for filtered

In [83]:
def fda_methods_batch_size(df, filename):
    pdf = PdfPages(filename)
    
    num_clients_values = sorted(df['num_clients'].unique())
    theta_values = sorted(df['theta'].unique())[1:]

    for num_clients in num_clients_values:
        for theta in theta_values:
            filtered_df = df[(df['theta'] == theta) & (df['num_clients'] == num_clients)] 
            synchronous_data = df[(df['num_clients'] == num_clients) & (df['fda_name'] == 'synchronous')] 
            
            fig, axs = plt.subplots(1, 2, figsize=(20, 6))
            
            for fda_name in fda_names:
                
                fda_data = filtered_df[filtered_df['fda_name'] == fda_name]
                
                if fda_data.empty:
                    continue

                axs[0].plot(fda_data['batch_size'], fda_data['common_channel_time_cost'], marker='o', label=fda_name, markersize=3)
                axs[1].plot(fda_data['batch_size'], fda_data['hypercube_time_cost'], marker='o', label=fda_name, markersize=3)
            
            if not synchronous_data.empty:
                axs[0].plot(synchronous_data['batch_size'], synchronous_data['common_channel_time_cost'], marker='o', label='synchronous', markersize=3)
                axs[1].plot(synchronous_data['batch_size'], synchronous_data['hypercube_time_cost'], marker='o', label='synchronous', markersize=3)

            
            if not axs[0].has_data():
                plt.close(fig)
                continue
            
            x_ticks = [32, 64, 128, 256]
            
            axs[0].set_xticks(x_ticks)
            axs[0].set_xlabel('Batch size')
            axs[0].legend()
            axs[0].set_title("Common Channel Communication Model")
            axs[0].set_ylabel('Time Cost')
            axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            axs[1].set_xticks(x_ticks)
            axs[1].set_xlabel('Batch size')
            axs[1].legend()
            axs[1].set_title("Hypercube Communication Model")
            axs[1].set_ylabel('Time Cost')
            axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            title = f'Num Clients : {num_clients} , $\Theta$ : {theta}'
            fig.suptitle(title)

            plt.tight_layout()
            pdf.savefig(fig)

            plt.close(fig)
    pdf.close()

In [84]:
def fda_methods_clients(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    theta_values = sorted(df['theta'].unique())[1:]

    for batch_size in batch_size_values:
        for theta in theta_values:
            
            filtered_df = df[(df['theta'] == theta) & (df['batch_size'] == batch_size)] 
            
            if filtered_df.empty:
                continue
                
            synchronous_data = df[(df['batch_size'] == batch_size) & (df['fda_name'] == 'synchronous')] 
            
            fig, axs = plt.subplots(1, 2, figsize=(20, 6))
            
            for fda_name in fda_names:
                
                fda_data = filtered_df[filtered_df['fda_name'] == fda_name]
                
                if fda_data.empty:
                    continue

                empty = False
                axs[0].plot(fda_data['num_clients'], fda_data['common_channel_time_cost'], marker='o', label=fda_name, markersize=3)
                axs[1].plot(fda_data['num_clients'], fda_data['hypercube_time_cost'], marker='o', label=fda_name, markersize=3)
            
            if not synchronous_data.empty:
                axs[0].plot(synchronous_data['num_clients'], synchronous_data['common_channel_time_cost'], marker='o', label='synchronous', markersize=3)
                axs[1].plot(synchronous_data['num_clients'], synchronous_data['hypercube_time_cost'], marker='o', label='synchronous', markersize=3)

            if not axs[0].has_data():
                plt.close(fig)
                continue
                
            # Set xticks every 5 units based on available values
            min_clients = 5
            max_clients = 60
            
            x_ticks = list(range(min_clients, max_clients + 1, 5))
            
            axs[0].set_xticks(x_ticks)
            axs[0].set_xlabel('Number of clients')
            axs[0].legend()
            axs[0].set_title("Common Channel Communication Model")
            axs[0].set_ylabel('Time Cost')
            axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            axs[1].set_xticks(x_ticks)
            axs[1].set_xlabel('Number of clients')
            axs[1].legend()
            axs[1].set_title("Hypercube Communication Model")
            axs[1].set_ylabel('Time Cost')
            axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            
            title = f'Batch Size : {batch_size} , $\Theta$ : {theta}'
            fig.suptitle(title)

            plt.tight_layout()

            pdf.savefig(fig)

            plt.close(fig)
    pdf.close()

# Help-Stat

In [85]:
def explore_top(df, acc_thresh, nn_name, fda_name):
    acceptable_acc_df = df[(df.accuracy > acc_thresh) & (df.nn_name == nn_name)]
    acceptable_acc_df = acceptable_acc_df[acceptable_acc_df['fda_name'] == fda_name]
    idx = acceptable_acc_df.groupby(['fda_name', 'num_clients', 'bias', 'batch_size', 'theta'], dropna=False)['epoch'].idxmin()
    filtered_acceptable_acc_df = acceptable_acc_df.loc[idx]
    return filtered_acceptable_acc_df[['num_clients', 'batch_size', 'theta', 'bias', 'total_rounds', 'total_fda_steps', 'epoch', 'total_communication_gb', 'cpu_time_cost', 'hypercube_time_cost', 'common_channel_time_cost']].sort_values(by='common_channel_time_cost')

In [86]:
def mean_epoch_per_method(df, acc_thresh, nn_name, fda_name):
    top_df = explore_top(df, acc_thresh, nn_name, fda_name)
    print(f"Mean epochs : {top_df['epoch'].mean()}")
    print(f"Std epochs : {top_df['epoch'].std()}")
    print(f"Mean rounds : {top_df['total_rounds'].mean()}")
    print(f"Std rounds : {top_df['total_rounds'].std()}")

## Save all those time-cost plots

In [87]:
import os

def time_cost_plots(df, acc_threshold, nn_name, limit_x_axis=False, show_runs=False, params=False, addi_name='', kde_time_log=True, kde_comm_log=True, kde_cpu_log=False,
                    show_bias=False, kde_time_log_bias=False):
    # Filter out based on `acc_threshold`
    acceptable_acc_df = df[(df.accuracy > acc_threshold) & (df.nn_name == nn_name)]
    
    str_thresh = str(acc_threshold).replace('.', '_')  # replace '.'
    
    if not os.path.exists(f"../../metrics/plots/{nn_name}/{str_thresh}"):
        os.makedirs(f"../../metrics/plots/{nn_name}/{str_thresh}")
    
    if show_runs:
        accept_acc_no_bias = acceptable_acc_df[acceptable_acc_df['bias'].isna()]
        # Plot the runs of each method. x-axis : time cost, y-axis : accuracy, PER number of clients (lines)
        fda_method_run_line_plot(accept_acc_no_bias, 'sketch', f"../../metrics/plots/{nn_name}/{str_thresh}/sketch_run.pdf", limit_x_axis=limit_x_axis)
        fda_method_run_line_plot(accept_acc_no_bias, 'naive', f"../../metrics/plots/{nn_name}/{str_thresh}/naive_run.pdf", limit_x_axis=limit_x_axis)
        fda_method_run_line_plot(accept_acc_no_bias, 'linear', f"../../metrics/plots/{nn_name}/{str_thresh}/linear_run.pdf", limit_x_axis=limit_x_axis)
        fda_method_run_line_plot(accept_acc_no_bias, 'synchronous', f"../../metrics/plots/{nn_name}/{str_thresh}/synchronous_run.pdf", limit_x_axis=limit_x_axis)
        
    # 1. Same runs not included
    
    # 2. Filter out same runs. We choose the instance which first hits the `acc_threshold`
    idx = acceptable_acc_df.groupby(['fda_name', 'num_clients', 'batch_size', 'theta', 'bias'], dropna=False)['epoch'].idxmin()
    filtered_acceptable_acc_df = acceptable_acc_df.loc[idx]
    
    # 2. NO BIAS. Same runs are NOT included
    no_bias_filt_df = filtered_acceptable_acc_df[filtered_acceptable_acc_df['bias'].isna()]
    kde_time_cost(no_bias_filt_df, f"../../metrics/plots/{nn_name}/{str_thresh}/kde_time_cost{addi_name}.pdf", x_log=kde_time_log)
    #scatter_time_cost_cpu_ratio(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/scatter_time_cost_cpu_ratio.pdf")
    kde_communication_cost(no_bias_filt_df, f"../../metrics/plots/{nn_name}/{str_thresh}/kde_communication_cost{addi_name}.pdf", x_log=kde_comm_log)
    kde_cpu_time_cost(no_bias_filt_df, f"../../metrics/plots/{nn_name}/{str_thresh}/kde_cpu_time_cost{addi_name}.pdf", x_log=kde_cpu_log)
    
    if show_bias:
        kde_time_cost_biases(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/biases.pdf", x_log=kde_time_log_bias)
    
    # Parameters fixed, and plot
    if params:
        fda_methods_batch_size(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/fda_methods_batch_size.pdf")
        fda_methods_clients(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/fda_methods_clients.pdf")

In [88]:
pd.set_option('display.max_rows', None)

In [89]:
df_bias = df[~df['bias'].isna()]
df_no_bias = df[df['bias'].isna()]

## AdvancedCNN

In [59]:
df_32 = df[
    (df['batch_size'] == 32) | (df['fda_name'] == 'synchronous')
]

df_fin = df_32[
    (df_32['theta'] >= 15) | (df_32['fda_name'] == 'synchronous')
]

In [60]:
time_cost_plots(df_fin, 0.99, 'AdvancedCNN', addi_name='_b32')
time_cost_plots(df_fin, 0.995, 'AdvancedCNN', addi_name='_b32')
time_cost_plots(df_fin, 0.996, 'AdvancedCNN', addi_name='_b32')

In [61]:
#explore_top(df_fin, 0.995, 'AdvancedCNN', 'synchronous')

In [44]:
explore_top(df_fin, 0.995, 'AdvancedCNN', 'naive')

,num_clients,batch_size,theta,bias,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,hypercube_time_cost,common_channel_time_cost
284408,5,32,100.0,NaN,24,3375,9,2.488581,29.878875,37.842336,45.805796
285357,5,32,50.0,NaN,45,3000,8,4.666024,26.559000,41.490276,56.421551
285460,5,32,75.0,NaN,40,4125,11,4.147606,36.518625,49.790963,63.063301
265822,10,32,100.0,NaN,28,4313,23,5.806705,38.182989,52.119081,79.991265
268026,15,32,100.0,NaN,23,3375,27,7.154680,29.878875,41.326363,83.300486
284505,5,32,10.0,NaN,119,2250,6,12.338927,19.919250,59.403815,98.888380
256569,5,256,1.5,NaN,138,938,20,14.308974,15.448860,61.237576,107.026292
269077,15,32,75.0,NaN,33,3500,28,10.265330,30.985500,47.410028,107.633297
285160,5,32,25.0,NaN,108,4125,11,11.198395,36.518625,72.353489,108.188354
304854,5,64,1.5,NaN,159,938,5,16.486423,9.684850,62.441405,115.197960


In [190]:
explore_top(df_fin, 0.996, 'AdvancedCNN', 'synchronous')

,num_clients,batch_size,theta,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,hypercube_time_cost,common_channel_time_cost
363720,20,128,0.0,4008,4008,171,1662.327299,48.051912,2042.844670,12681.739381
394045,5,256,0.0,23250,23250,496,2410.747860,382.927500,8097.320652,15811.713804
401789,15,64,0.0,8750,8750,140,2721.812100,90.343750,4445.243110,20413.207430
399117,35,32,0.0,3643,3643,68,2644.149728,32.251479,2449.759802,20581.072223
363097,15,128,0.0,17125,17125,548,5326.975110,205.311625,8728.471801,39980.059113
397692,10,32,0.0,26813,26813,143,5560.376978,237.375489,13582.280236,40272.089731
378740,20,256,0.0,13958,13958,1191,5789.112883,229.888260,7176.823719,44227.146167
401493,10,64,0.0,32250,32250,344,6687.881160,332.981250,16383.896034,48485.725602
365238,25,128,0.0,12919,12919,689,6697.731528,154.885891,8727.982246,51593.464023
377071,15,256,0.0,23782,23782,1522,7397.729756,391.689540,12228.057149,55628.071716


In [191]:
explore_top(df_fin, 0.996, 'AdvancedCNN', 'naive')

,num_clients,batch_size,theta,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,hypercube_time_cost,common_channel_time_cost
285367,5,32,50.0,97,6750,18,10.057879,59.757750,91.942962,124.128174
268053,15,32,100.0,43,6750,54,13.376167,59.757750,81.159618,159.633133
269101,15,32,75.0,58,6500,52,18.042116,57.544500,86.411885,192.258966
266367,10,32,15.0,137,3375,18,28.410669,29.878875,98.064480,234.435691
285500,5,32,75.0,163,19125,51,16.901540,169.313625,223.398552,277.483478
270705,20,32,30.0,108,5250,56,44.793671,46.478250,100.230655,386.910146
268688,15,32,20.0,151,4875,39,46.970993,43.158375,118.311963,393.875121
270598,20,32,25.0,125,4594,49,51.844408,40.670682,102.883971,434.688179
274104,30,32,25.0,94,3438,55,58.480490,30.436614,92.815803,482.685734
285210,5,32,25.0,504,22875,61,52.259250,202.512375,369.741974,536.971574


In [353]:
#explore_top(df_fin, 0.995, 'AdvancedCNN', 'linear')

## LeNet-5

In [44]:
explore_top(df_no_bias, 0.985, 'LeNet-5', 'synchronous')

,num_clients,batch_size,theta,bias,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,hypercube_time_cost,common_channel_time_cost
1025879,5,256,0.0,NaN,1407,1407,30,3.472814,13.026006,24.139010,35.252014
980891,10,256,0.0,NaN,985,985,42,4.862433,9.119130,20.788969,44.128646
976374,5,128,0.0,NaN,2344,2344,25,5.785555,18.798880,37.312655,55.826429
1054463,5,64,0.0,NaN,2625,2625,14,6.479130,19.711125,40.444341,61.177557
954382,10,128,0.0,NaN,1547,1547,33,7.636735,12.406940,30.735103,67.391429
985429,15,256,0.0,NaN,1250,1250,80,9.255900,11.572500,26.381940,80.683220
956645,15,128,0.0,NaN,1438,1438,46,10.647987,11.532760,28.569540,91.037732
1044674,10,64,0.0,NaN,2344,2344,25,11.571109,17.601096,45.371758,100.913082
989954,20,256,0.0,NaN,1231,1231,105,12.153614,11.396598,25.980935,103.764063
958905,20,128,0.0,NaN,1313,1313,56,12.963196,10.530260,26.086096,109.050553


In [45]:
explore_top(df_no_bias, 0.985, 'LeNet-5', 'naive')

,num_clients,batch_size,theta,bias,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,hypercube_time_cost,common_channel_time_cost
721081,5,256,1.0,NaN,41,1500,32,0.101228,13.887000,14.210929,14.534858
644111,10,256,1.5,NaN,37,1454,62,0.182708,13.461132,13.899631,14.776629
642110,10,256,0.5,NaN,78,1430,61,0.385103,13.238940,14.163186,16.011679
599101,15,128,1.0,NaN,54,1625,52,0.399952,13.032500,13.672424,16.018811
649145,15,256,1.0,NaN,56,1500,96,0.414754,13.887000,14.550607,16.983832
665196,25,256,1.0,NaN,55,1379,147,0.678904,12.766782,13.635779,17.980764
598102,15,128,0.5,NaN,85,1657,53,0.629501,13.289140,14.296341,17.989411
635072,5,128,1.0,NaN,53,2157,23,0.130860,17.299140,17.717892,18.136643
652161,15,256,1.5,NaN,42,1750,112,0.311103,16.201500,16.699265,18.524404
647145,15,256,0.5,NaN,84,1500,96,0.622086,13.887000,14.882338,18.531912


In [90]:
explore_top(df_no_bias[df_no_bias['batch_size'] == 32], 0.985, 'LeNet-5', 'naive') #~~~

,num_clients,batch_size,theta,bias,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,hypercube_time_cost,common_channel_time_cost
704518,10,32,3.0,NaN,35,3563,19,0.172919,23.562119,23.977125,24.807138
703316,10,32,0.5,NaN,149,3188,17,0.735663,21.082244,22.847835,26.379018
705226,15,32,1.0,NaN,94,3375,27,0.696246,22.318875,23.432869,27.517513
720410,5,32,1.0,NaN,101,4125,11,0.249375,27.278625,28.076624,28.874623
703520,10,32,1.0,NaN,107,3938,21,0.528361,26.041994,27.310060,29.846192
703921,10,32,1.5,NaN,78,4125,22,0.385210,27.278625,28.203130,30.052140
707943,20,32,3.0,NaN,43,4125,44,0.424867,27.278625,27.788466,30.507616
709047,25,32,1.5,NaN,71,3600,48,0.876585,23.806800,24.928829,30.538974
705631,15,32,1.5,NaN,74,4000,32,0.548189,26.452000,27.329103,30.545147
705026,15,32,0.5,NaN,158,3375,27,1.170148,22.318875,24.191112,31.055982


In [60]:
explore_top(df, 0.985, 'LeNet-5', 'naive')

,num_clients,batch_size,theta,bias,total_rounds,total_fda_steps,epoch,total_communication_gb,cpu_time_cost,hypercube_time_cost,common_channel_time_cost
712781,5,256,1.0,NaN,41,1500,32,0.101228,13.887000,14.210929,14.534858
635811,10,256,1.5,NaN,37,1454,62,0.182708,13.461132,13.899631,14.776629
633810,10,256,0.5,NaN,78,1430,61,0.385103,13.238940,14.163186,16.011679
590801,15,128,1.0,NaN,54,1625,52,0.399952,13.032500,13.672424,16.018811
640845,15,256,1.0,NaN,56,1500,96,0.414754,13.887000,14.550607,16.983832
656896,25,256,1.0,NaN,55,1379,147,0.678904,12.766782,13.635779,17.980764
589802,15,128,0.5,NaN,85,1657,53,0.629501,13.289140,14.296341,17.989411
626772,5,128,1.0,NaN,53,2157,23,0.130860,17.299140,17.717892,18.136643
643861,15,256,1.5,NaN,42,1750,112,0.311103,16.201500,16.699265,18.524404
638845,15,256,0.5,NaN,84,1500,96,0.622086,13.887000,14.882338,18.531912


In [216]:
#explore_top(df, 0.98, 'LeNet-5', 'linear')

In [209]:
time_cost_plots(df, 0.98, 'LeNet-5', kde_time_log=False, kde_comm_log=True, show_bias=True, kde_time_log_bias=True)
time_cost_plots(df, 0.975, 'LeNet-5', kde_time_log=False, kde_comm_log=True, show_bias=True, kde_time_log_bias=True)
time_cost_plots(df, 0.985, 'LeNet-5', kde_time_log=False, kde_comm_log=True, show_bias=True, kde_time_log_bias=True)
#time_cost_plots(df, 0.987, 'LeNet-5', kde_time_log=False, kde_comm_log=True, show_bias=True)
#time_cost_plots(df, 0.99, 'LeNet-5', kde_time_log=False, kde_comm_log=True, show_bias=True)

In [168]:
mean_epoch_per_method(df, 0.98, 'LeNet-5', 'synchronous')

Mean epochs : 63.666666666666664
Std epochs : 45.50489023677557
Mean rounds : 1274.9375
Std rounds : 447.0183206417402


In [169]:
mean_epoch_per_method(df_fin, 0.995, 'AdvancedCNN', 'naive')

Mean epochs : 40.266666666666666
Std epochs : 24.717186851312523
Mean rounds : 89.15555555555555
Std rounds : 76.19547763145556


In [170]:
mean_epoch_per_method(df, 0.995, 'AdvancedCNN', 'synchronous')

Mean epochs : 192.22222222222223
Std epochs : 310.4632088354497
Mean rounds : 3307.0666666666666
Std rounds : 1897.6396750604779
